## Load image

In [77]:
import cv2 as cv
import numpy as np 

In [78]:
# image = "./images/test1.jpg"
image = "./images/test2-nored.jpg"
highlighted_image = "./images/test2.jpg"
output = "./images/removed2.png"

In [79]:
watermarked = cv.imread(image)
# display
def display(name, img):
    cv.startWindowThread()
    cv.imshow(name, img/255)
    cv.waitKey(0)
    cv.destroyWindow(name)
    cv.waitKey(1)

display("Watermarked", watermarked)


## Detect watermark 

*Note*: This is only for white semitransparent watermark. 

Find the pixels that are in a range of white after converting the image to greyscale. To decrease noise, the user enters the ROI of the watermark. Then, compute an alpha mask of the watermark. 

In [80]:
# obtain the alpha mask

# rectangle to identify watermark
# test1  
# x, y, w, h = 600, 1400, 250, 35 
# test2 
x, y, w, h = 580, 1480, 470, 70 
# test3 
# x, y, w, h = 600, 1400, 250, 35 

selected = np.zeros_like(watermarked)
cv.rectangle(selected, (x,y), (x+w, y+h), (255,255,255), -1)

# for lighter images (test2 and test3)
highlighted = cv.imread(highlighted_image) # highlighted with red strokes 
watermark = cv.bitwise_and(highlighted, selected) 
watermark = watermark.astype(float)
# display("Selected", watermark)
watermark[np.where((watermark != [0,0,255]).all(axis = 2))] = [0,0,0] # and all other pixels to black 
# watermark[np.where((watermark == [0,0,255]).all(axis = 2))] = watermarked[np.where((watermark == [0,0,255]).all(axis = 2))] # convert red to white 


grey = watermark.max(axis=-1)
grey[grey>0] = 255
display("Mask", grey)

# HSV (alternative)
# hsv = cv.cvtColor(watermark, cv.COLOR_BGR2HSV)
# sensitivity = 50
# lower_white = np.array([0, 0, 255-sensitivity])
# upper_white = np.array([255, sensitivity, 255])
# hsv_mask = cv.inRange(hsv, lower_white, upper_white)
# display("Mask", hsv_mask)


## Calculate alpha and W 


In [81]:
# for light images 
W = grey 
alpha = np.zeros_like(W)
alpha[W>0] = 1 # alpha is 1 

alpha = np.repeat(alpha, 3, axis=-1).reshape(*(alpha.shape),3)
W = np.repeat(W, 3, axis=-1).reshape(*(W.shape),3)

In [82]:
# NOT USED 
def blur_mask(watermarked, alpha, W):
    I = (watermarked.astype(float) - alpha * W)/(1-alpha)
    I[I<0] = 0 
    I[I>255] = 255 
    I = I.astype(uint8)

    fI = I.copy()
    fI[y:y+h, x:x+w, :] = cv.medianBlur(fI[y:y+h, x:x+w, :], 5) 

 

## Main logic 
J = alpha * W + (1-alpha) * I 

I = \frac{J - alpha * W}{1 - alpha}

In [83]:
def process(watermarked, alpha, W, bg, output_name="removed.png"):
    aW = (alpha * W)
    # a1 = (1 / (1-alpha))
    med = bg 
    watermarked = watermarked.astype(float)
    J = watermarked

    # display("Watermarked", J)
    I = (J - aW) # * a1, but for test1, alpha is around 1
    I[I<0] = 0
    I[I>255] = 255 
    I = I.astype(np.uint8)

    display("Output", I) 


    # blur 
    fI = I.copy() 
    fI[y:y+h, x:x+w, :] = cv.medianBlur(fI[y:y+h, x:x+w, :], 7) 
    fI[fI<0] = 0
    fI[fI>255] = 255 

    display("Output2", fI)

        # predict the color from the left and top neighbor 
    for row in range(y, y+h):
        for col in range(x, x+w):
            if fI[row, col][0] == 0 and fI[row, col][1] == 0 and fI[row, col][2] == 0:
                crop = fI[row-1:row, col-1:col]
                average = np.average(crop, axis=(0,1))
                fI[row, col] = average # + I[row, col-1] 

    display("Output test", fI)

    fI[y:y+h, x:x+w, :] = cv.medianBlur(fI[y:y+h, x:x+w, :], 7) 
    fI[fI<0] = 0
    fI[fI>255] = 255

    display("Test", fI)

    cv.imwrite(output_name, fI)

In [84]:
process(watermarked, alpha, W, W, output)